# 公众号：早起Python

In [8]:
import requests
import re
from bs4 import BeautifulSoup
import time
import datetime
import random
import pandas as pd
from urllib import request, parse
import warnings
import json
warnings.filterwarnings("ignore")

# 爬取天气相关代码

# 下面为杭州天气，更换城市请自行修改天气网对应url

In [2]:
r = requests.get('https://www.tianqi.com/hangzhou/')
soup = BeautifulSoup(r.text)

date = soup.find_all('dd',class_ = 'week')[0].text.strip()
date = re.sub(u'\u3000', "，", date)
time_now = time.strftime("%H:%M:%S", time.localtime()) 

weather =soup.select('dd > span')[0].text

air = soup.select('dd > h5')[0].text

time1 = datetime.datetime.strptime(time_now,"%H:%M:%S")
time2 = datetime.datetime.strptime('9:30:00',"%H:%M:%S")
time3 = round((time2 - time1).total_seconds() / 60)

In [3]:
print(date)
print(time_now)
print(weather)
print(air)

2021年02月13日，星期六，辛丑年正月初二
07:52:42
阴8 ~ 16℃
空气质量：优


# 交通相关

# 请务必按照原文中的方式在百度地图开放平台申请一个AK后方能执行下面代码！！！！！

In [5]:
AK = "替换为你申请的AK" 

def get_position(name,AK):
    url = f'http://api.map.baidu.com/geocoding/v3/?address={name}&output=json&ak={AK}'
    res = requests.get(url)
    val = res.json()
    retval = {'地址':name,'经度':val['result']['location']['lng'],'纬度':val['result']['location']['lat'],'地区标签':val['result']['level'],'是否精确查找':val['result']['precise']}
    longitude = retval['经度']
    latitude = retval['纬度']

    return(longitude,latitude)

In [6]:
get_position('杭州市益乐新村',AK)

(120.11614538041329, 30.294761455511946)

## 复制上面的经纬度构造URL

# 1、驾车信息相关

In [9]:

url = f'http://api.map.baidu.com/directionlite/v1/driving?origin=40.01116,116.339303&destination=39.936404,116.452562&ak={AK}'
data = requests.get(url)
data = data.content.decode('unicode_escape')
data = json.loads(data)

distance = data['result']['routes'][0]['distance']/1000 #驾车距离
time_cost = round(data['result']['routes'][0]['duration']/60) #驾车耗时

lukuang = data['result']['routes'][0]['traffic_condition']
lukuang = '畅通' if lukuang == 1 else '缓行' if lukuang == 2 else '拥堵' if lukuang == 3 else '严重拥堵' #路况

In [10]:
data

{'status': 0,
 'message': 'ok',
 'result': {'origin': {'lng': 116.339303, 'lat': 40.011160069003},
  'destination': {'lng': 116.452562, 'lat': 39.93640407532},
  'routes': [{'distance': 17492,
    'duration': 1720,
    'traffic_condition': 1,
    'toll': 0,
    'steps': [{'leg_index': 0,
      'distance': 128,
      'duration': 10,
      'direction': 6,
      'turn': 7,
      'road_type': 6,
      'road_types': '6',
      'instruction': '从<b>起点</b>向正南方向出发,行驶130米,<b>左转</b>',
      'path': '116.33964557069,40.01051873416;116.33967584359,40.010168642615;116.33970647581,40.009358617237',
      'traffic_condition': [{'status': 0, 'geo_cnt': 2}],
      'start_location': {'lng': '116.33964557069', 'lat': '40.01051873416'},
      'end_location': {'lng': '116.33970647581', 'lat': '40.009358617237'}},
     {'leg_index': 0,
      'distance': 330,
      'duration': 32,
      'direction': 3,
      'turn': 3,
      'road_type': 6,
      'road_types': '6',
      'instruction': '行驶330米,<b>右转</b>进入<b>荷

In [11]:
lukuang

'畅通'

# 2、 公交相关

In [13]:
#公交地铁
url = f'http://api.map.baidu.com/directionlite/v1/transit?origin=40.01116,116.339303&destination=39.936404,116.452562&ak={AK}' 
data1 = requests.get(url)
data1 = data1.content.decode('unicode_escape')
data1 = json.loads(data1)

time_cost1 = round(data1['result']['routes'][0]['duration']/60) #驾车耗时

In [14]:
data1

{'status': 0,
 'message': 'ok',
 'result': {'origin': {'lng': 116.33929505188, 'lat': 40.011157363344},
  'destination': {'lng': 116.45255341058, 'lat': 39.936401378723},
  'routes': [{'distance': 21421,
    'duration': 5342,
    'price': 7,
    'line_price': [{'line_price': 5, 'line_type': 1},
     {'line_price': 2, 'line_type': 0}],
    'steps': [[{'distance': 1159,
       'duration': 989,
       'type': 5,
       'instruction': '步行1.2公里',
       'vehicle': {'direct_text': '',
        'name': '',
        'line_id': '',
        'start_name': '',
        'end_name': '',
        'start_time': '',
        'end_time': '',
        'stop_num': 0,
        'total_price': 0,
        'type': 0,
        'zone_price': 0},
       'path': '116.33996878101,40.011178086743;116.34000471324,40.010542566293;116.34002267935,40.010169540643;116.34006759462,40.009368219334;116.34052573043,40.009375127317;116.3407323407,40.009375127317;116.3409120018,40.009361311351;116.34188217175,40.00933367941;116.342376

# 3、上车点

In [15]:
#上车点

url = f'http://api.map.baidu.com/parking/search?location=120.11614538041329,30.294761455511946&coordtype=bd09ll&ak={AK}'
data2 = requests.get(url)

num = len(json.loads(json.loads(json.dumps(data2.text, ensure_ascii=False)))['recommendStops'])


data2 =json.loads(json.loads(json.dumps(data2.text, ensure_ascii=False)))['recommendStops']


shangche = []

for i in range(num):
    
    shangche.append(data2[i]['name'])

    
shangche = '、'.join(shangche)

In [16]:
data2

[{'name': '益乐新村北区-南1门',
  'address': '',
  'distance': 173.51348876953,
  'bd09ll_x': 120.11534921223978,
  'bd09ll_y': 30.29335633925444,
  'gcj02ll_x': 120.10891310104522,
  'gcj02ll_y': 30.287052696697216,
  'id': '63bf6a4b8ba88603d4b74ea0'},
 {'name': '西斗门路丰潭路口-公交站-路北附近',
  'address': '',
  'distance': 206.71768188477,
  'bd09ll_x': 120.1147922628236,
  'bd09ll_y': 30.296209583345718,
  'gcj02ll_x': 120.1083570755251,
  'gcj02ll_y': 30.289912468862674,
  'id': '3e3678e155dfe281ae080fe6'},
 {'name': '中国美术学院风景建筑设计研究总院有限公司停车场-入口',
  'address': '',
  'distance': 250.67141723633,
  'bd09ll_x': 120.11799023043912,
  'bd09ll_y': 30.296349904662318,
  'gcj02ll_x': 120.11156284172019,
  'gcj02ll_y': 30.290022384119958,
  'id': 'b62370ff73be90c579b55cc9'}]

# 喵提醒发送至微信

# 请务必申请一个喵提醒ID，[点击直达教程地址](https://mp.weixin.qq.com/s?__biz=MzIxNjQ1NzU2Mw==&mid=100000042&idx=6&sn=c9656c8b242972f62fe5840ddf7487db&scene=19#wechat_redirect)

In [17]:
text = f"""

哈喽，冰冰，早上好!

今天是{date}。

现在已经{time_now}啦，距离上班还有{time3}分钟。

外面的天气情况不是很好，{weather}，{air}，记得带伞哦～

根据现在的路况，路况{lukuang}，总共{distance}千米，打车去公司需要{time_cost}分钟，坐公交地铁的话需要{time_cost1}分钟。

如果需要打车去公司的话，最近的上车点为{shangche}。

加油，打工人，再上一天班就放假咯！

"""
print(text)



哈喽，冰冰，早上好!

今天是2021年02月13日，星期六，辛丑年正月初二。

现在已经07:52:42啦，距离上班还有97分钟。

外面的天气情况不是很好，阴8 ~ 16℃，空气质量：优，记得带伞哦～

根据现在的路况，路况畅通，总共17.492千米，打车去公司需要29分钟，坐公交地铁的话需要89分钟。

如果需要打车去公司的话，最近的上车点为益乐新村北区-南1门、西斗门路丰潭路口-公交站-路北附近、中国美术学院风景建筑设计研究总院有限公司停车场-入口。

加油，打工人，再上一天班就放假咯！




In [18]:
your_id = "替换成你申请的ID"
class Message(object):
    def __init__(self,text):
        self.text = text
    def push(self):
        # id 为相应提醒的喵码
        page = request.urlopen("http://miaotixing.com/trigger?" + parse.urlencode({"id": your_id, "text": self.text, "type": "json"}))
        result = page.read()
        jsonObj = json.loads(result)
        if (jsonObj["code"] == 0):
            print("\nReminder message was sent successfully")
        else:
            print("\nReminder message failed to be sent，wrong code：" + str(jsonObj["code"]) + "，describe：" + jsonObj["msg"])


message = Message(text)
message.push() # 完成推送


Reminder message was sent successfully
